In [9]:
%load_ext autoreload
%autoreload 2
import numpy as np
from matplotlib import pyplot as plt
import reg_helper as RHelper
import pandas as pd
from data_helper import get_data
import optuna

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
X_train, y_train, X_val, y_val = get_data()

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
def objective(trial):
    hidden_units = trial.suggest_int("hidden_units", 5, 200)
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 1e1, log=True)
    activation = 'relu'
    optimizer = 'adam'

    model = Sequential()
    model.add(Dense(hidden_units, input_shape=(2,), activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    es = EarlyStopping(monitor='val_acc', verbose=1, patience=6, mode='max', 
                       restore_best_weights=True)
    if optimizer=='adam':
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = SGD(learning_rate=learning_rate)
    model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=len(X_train), epochs=100, verbose=0, 
              validation_data=(X_val, y_val), callbacks=[es])
    acc = np.array(model.evaluate(X_val, y_val, verbose=0)[0])
    return acc

In [23]:
study = optuna.create_study(
    # direction="maximize"
    direction="minimize"
)
study.optimize(objective, n_trials=100)


[I 2024-09-19 16:04:22,708] A new study created in memory with name: no-name-0ec114e5-c3a6-4105-bd1d-ca0b3fe631ca
[I 2024-09-19 16:04:24,547] Trial 0 finished with value: 0.5452671647071838 and parameters: {'hidden_units': 55, 'learning_rate': 0.41763553281919746}. Best is trial 0 with value: 0.5452671647071838.
[I 2024-09-19 16:04:26,251] Trial 1 finished with value: 0.5201385021209717 and parameters: {'hidden_units': 173, 'learning_rate': 0.35734162285566423}. Best is trial 1 with value: 0.5201385021209717.
[I 2024-09-19 16:04:27,927] Trial 2 finished with value: 0.40057721734046936 and parameters: {'hidden_units': 156, 'learning_rate': 0.018712954957202085}. Best is trial 2 with value: 0.40057721734046936.
[I 2024-09-19 16:04:29,581] Trial 3 finished with value: 5.04437780380249 and parameters: {'hidden_units': 115, 'learning_rate': 6.093259127198592}. Best is trial 2 with value: 0.40057721734046936.
[I 2024-09-19 16:04:31,214] Trial 4 finished with value: 0.3798951506614685 and par

In [24]:
trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Accuracy: 0.33683672547340393
Best hyperparameters: {'hidden_units': 15, 'learning_rate': 0.013999536188394463}


In [25]:
optuna.visualization.plot_optimization_history(study)

In [26]:
optuna.visualization.plot_contour(study, params=["hidden_units", "learning_rate"])